## 필요 패키지 설치

허깅페이스의 트랜스포머스 라이브러리를 설치합니다.

In [ ]:
!pip install --upgrade transformers

In [61]:
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-gty44fsg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-gty44fsg
  Resolved https://github.com/huggingface/accelerate to commit 649e65b542a5740fb5ce663bbd5af45ed426c06f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.27.0.dev0-py3-none-any.whl size=273101 sha256=26f1a62f265be15b5962819743dfe5f977bc5c108de6cc194439055e1dede44d
  Stored in directory: /tmp/pip-ephem-wheel-cache-q4mqkrbx/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.26.1
    Uninstalling accelerate-0.26.1:
      Successfully uninstalled accelerate-0.26.1


데이터 셋을 다운받는기 위해 다음 명령을 실행해서 허깅페이스 `Datasets` 라이브러리를 설치합니다.

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


그리고 T5 모델의 토크나이저가 `sentencepiece`를 사용하므로 다음을 실행해서 설치합니다.

In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.4 MB/s eta 0:00:00


또 모델 평가를 위해 허깅페이스 `evaluate` 라이브러리와 BLEU 점수를 계산하기위해 `sacrebleu`를 설치합니다.

In [4]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00


In [5]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.3 MB/s eta 0:00:00


## Data Set

In [1]:
# 데이터 셋을 다운받을 함수를 임포트 합니다.
from datasets import load_dataset

In [2]:
# 데이터을 다운로드 합니다.
dataset = load_dataset("Moo/korean-parallel-corpora")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


데이터를 확인해보면 `DatasetDict`라는 것을 알 수 있고 안에 `train`, `valid`, `test` 키가 있고, 각키 안에 `en`과 `ko`가 있는 것을 확인할 수 있습니다.

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ko', 'en'],
        num_rows: 96215
    })
    validation: Dataset({
        features: ['ko', 'en'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['ko', 'en'],
        num_rows: 2000
    })
})

In [4]:
# 데이터 셋을 pandas로 바꿔주기 위해 pandas를 임포드 합니다.
import pandas as pd

In [5]:
# 학습을 위해 활용할 데이터 프레임을 만들어 줍니다.
# 이때, 빠르게 결과값을 보기 위해 학습 데이터는 1000개, 검증, 평가 데이터는 100개의 데이터만 학습에 활용합니다.
# (성능이 더 좋은 모델을 만들려면 잘 정제된 더 많은 데이터를 활용하면 됩니다.)
train = pd.DataFrame()

train['ko'] = dataset['train']['ko'][:1000]
train['en'] = dataset['train']['en'][:1000]

In [6]:
valid = pd.DataFrame()

valid['ko'] = dataset['validation']['ko'][:100]
valid['en'] = dataset['validation']['en'][:100]

In [7]:
test = pd.DataFrame()

test['ko'] = dataset['test']['ko'][:100]
test['en'] = dataset['test']['en'][:100]

In [8]:
# 필요할 때 파일을 읽어 허깅페이스 데이터 셋을 만들어 활용하기 위해 정데한 데이터 셋을 저장합니다.
train.to_csv("train.csv", index=False)
valid.to_csv("valid.csv", index=False)
test.to_csv("test.csv", index=False)

필요할때 이 파일을 읽어 허깅페이스 데이터셋을 만들 수 있습니다.

아래처럼 데이터 셋과 데이터 셋의 위치를 정의한 사전을 `load_dataset`에 넘기면 됩니다.

In [9]:
data_files = {"train": "train.csv", "valid": "valid.csv", "test": "test.csv"}

In [10]:
dataset =  load_dataset("csv", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

제대로 로딩되었는지 `dataset`을 확인해봅니다.

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ko', 'en'],
        num_rows: 1000
    })
    valid: Dataset({
        features: ['ko', 'en'],
        num_rows: 100
    })
    test: Dataset({
        features: ['ko', 'en'],
        num_rows: 100
    })
})

`DatasetDict`에 `train`, `valid`, `test` 키로 1000 문장, 100 문장, 100 문장이 저장된 것을 확인할 수 있다.

이 데이터 셋에서 개별 샘플에 대한 접근은 `[split][feature][row num]` 형태로 가능하다.

In [12]:
# train 스플릿에서 영어 3개와 한국어 3개 샘플을 가져옵니다.
print(dataset['train']['en'][:3])
print(dataset['train']['ko'][:3])

['Much of personal computing is about "can you top this?"', 'so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.', "Like all optical mice, But it also doesn't need a desk."]
['개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"', '모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.', '그러나 이것은 또한 책상도 필요로 하지 않는다.']


이때, `feature`와 `row num`은 순서를 바꿔서 사용할 수 도 있습니다.

In [13]:
print(dataset['train'][:3]['en'])
print(dataset['train'][:3]['ko'])

['Much of personal computing is about "can you top this?"', 'so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.', "Like all optical mice, But it also doesn't need a desk."]
['개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"', '모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.', '그러나 이것은 또한 책상도 필요로 하지 않는다.']


데이터를 어떻게 조회하는지는 데이터 구성 방식에 따라 조금씩 다르므로 데이터 셋을 보고 몇번 해보면 금방 접근법을 알 수 있다.

## Hugging face

데이터 셋 준비를 마쳤으니 학습을 진행해 봅니다. 허깅페이스에서 제공하는 필요 클래스를 임포트 합니다.

먼저 사전학습된 모델을 사용하기 위한 클래스를 임포트 합니다. `AutoTokenizer`는 사전학습된 모델이 사용한 토크나이저를 읽기 위해 필요하며 `AutoModelForSeq2SeqLM`은 시퀀스 투 스퀀스 방식으로 작동하는 사전학습된 모델을 불러 올 때 마지막에 분류기 헤드를 붙여서 모델을 로딩하기 위해 사용합니다.

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

다음은 데이터 콜레이터를 임포트합니다. 데이터 단위 들을 적절히 정리해서 모델에 입력하는 작업이 필요합니다. 이 과정은 `DataCollatorForSeq2Seq`가 자동으로 처리하게 됩니다.

In [15]:
from transformers import DataCollatorForSeq2Seq

그리고 학습에 필요한 클래스를 임포트 합니다. 학습에 필요한 설정을 `Seq2SeqTrainingArguments`에 정의하고 실제 학습은 `Seq2SeqTrainer`로 하게 됩니다.
`Seq2SeqTrainer`는 `generate()`함수를 제공합니다.

In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

데이터 셋을 로딩하는 함수와 번역 결과를 측정할 함수를 로딩합니다.

In [17]:
from datasets import load_dataset, load_metric

그외 필요한 라이브러리들을 임포트 합니다.

In [18]:
import numpy as np
import torch
import multiprocessing

허깅페이스에서 파이토치 기반 구현을 사용하므로 gpu가 있다면 `device`를 세팅합니다.

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

미리 학습된 모델의 체크포인트를 세팅합니다. 여기서 사용할 모델은 한국어와 영어에 미리 학습된 [KE-T5](https://github.com/AIRC-KETI/ke-t5)모델을 사용합니다. T5모델은 트랜스포머의 인코더, 디코더 구조를 모두 사용하는 모델로 번역기를 만들 때 사용할 수 있는 모델이다. 아래처럼 모델 체크 포인트와 T5 모델에 입력될 최대 토큰 길이를 설정합니다.

In [20]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

## Tokenizer

먼저 모델 체크 포인트를 사용하여 KE-T5 모델이 학습할때 함께 사용한 토크나이저를 불러옵니다.

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


토크나이저를 불러왔으니 현재 사용하는 데이터셋에서 샘플을 가져와 토크나이징을 진행해 봅니다.

In [22]:
dataset['train'][1]['en'], dataset['train'][1]['ko']

('so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.',
 '모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.')

토크나이저에 각 문장을 입력하면 토큰화된 결과를 확인할 수 있습니다.

In [23]:
tokenized_sample_en = tokenizer(dataset['train'][1]['en'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_en

{'input_ids': [262, 16, 10239, 16, 1051, 2520, 1590, 155, 16, 742, 36458, 1191, 16787, 54880, 45836, 3866, 20, 926, 742, 36458, 1191, 4, 16787, 45836, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
tokenized_sample_ko = tokenizer(dataset['train'][1]['ko'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_ko

{'input_ids': [350, 2908, 56715, 41, 10165, 7, 34, 47, 2908, 56715, 27, 14740, 2709, 30882, 50904, 7, 25963, 21, 11995, 751, 985, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

문장에 토큰으로 쪼개지고 각 토큰이 숫자로 변환된 것을 볼 수 있습니다. 이렇게 숫자화된 토큰을 `input_ids`로 반환하고 추가로 트랜스포머 인코더, 디코더에 쓰일 패딩 마스크도 함께 `attention_mask`로 돌려줍니다. 마스크가 모두 1인 이유는 샘플이 하나밖에 없기 때문입니다.

샘플 몇개를 더 실험해보겠습니다.

In [25]:
tokenizer(dataset['train'][:3]['en'],
          max_length=max_token_length,
          padding=True, truncation=True)

{'input_ids': [[24035, 14, 3109, 24291, 43, 155, 23, 12578, 106, 1081, 110, 656, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [262, 16, 10239, 16, 1051, 2520, 1590, 155, 16, 742, 36458, 1191, 16787, 54880, 45836, 3866, 20, 926, 742, 36458, 1191, 4, 16787, 45836, 3, 1], [8513, 174, 54880, 50453, 4, 341, 60, 187, 1686, 17, 71, 713, 16, 23899, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

샘플 3개 중 최대 길이를 갖는 샘플에 맞춰서 패딩되는 모습을 확인할 수 있습니다.

실제로 dataset['train'][1]['en']에 대해서 어떻게 토큰화 되었는지 확인해보겠습니다.

In [26]:
pd.DataFrame(
    [
        tokenized_sample_en['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_en['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
ids,262,16,10239,16,1051,2520,1590,155,16,742,...,20,926,742,36458,1191,4,16787,45836,3,1
tokens,▁so,▁a,▁mention,▁a,▁few,▁weeks,▁ago,▁about,▁a,▁re,...,▁in,▁another,▁re,charge,able,",",▁wireless,▁mouse,.,</s>


실제로 dataset['train'][1]['ko']에 대해서 어떻게 토큰화 되었는지 확인해보겠습니다.

In [27]:
pd.DataFrame(
    [
        tokenized_sample_ko['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_ko['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
ids,350,2908,56715,41,10165,7,34,47,2908,56715,...,30882,50904,7,25963,21,11995,751,985,3,1
tokens,▁모든,▁광,마우스,와,▁마찬가지,▁,로,▁이,▁광,마우스,...,▁놓는,▁마우스,▁,패드,를,▁필요로,▁하지,▁않는다,.,</s>


KE-T5를 학습할때 학습된 규칙대로 토큰화가 진행됩니다. 영어에서 `rechargeable`은 `charge`, `able`으로 쪼개진 것을 볼 수 있고, 한국어에서 `광마우스`은 `광`, `마우스`로 쪼개진것을 볼 수 있습니다. 토큰 앞에 _표시는 이 토큰 앞에는 공백이 있어야 한다는 의미입니다. 그리고 마지막에 엔드 토큰인 `</s>`가 항상 붙게 되는 것도 확인할 수 있습니다.

이제 앞서 csv파일로 부터 로딩한 `dataset`내의 문장을 모두 토크나이저를 사용해서 숫자로 바꾸는 작업을 해야 합니다. 즉 문자로된 문장을 숫자로 바꿔 특성화 해야 합니다. `dataset.map()`함수에 각 샘플을 토큰화 하는 함수를 만들어 전달하면 `map()`이 모든 샘플에 대해 전달받은 함수를 적용하게 되는데 함수는 아래와 같이 작성합니다.

In [28]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['en'],
                             text_target=examples['ko'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

`convert_examples_to_features()`가 하고 싶은 일은 `dataset`에 있는 샘플 문장을 정수로 바꿔줍니다. `convert_examples_to_features()`가 `dataset`에 적용될 때 넘겨 받는 `examples`는 다음과 같이 넘어 옵니다.

```python
examples= {'en':['sent1', 'sent2', ... , 'sent10'], # 이건 문장 10개짜리 리스트
           'ko':['sent1', 'sent2', ... , 'sent10']}
```

그리고 `remove_columns` 인자에 기존 특성 이름인 `en`, `ko`를 전달해서 기존 특성은 제거하게 한다. 이 특성이 있으면 이후 콜레이터가 샘플들을 미니 배치로 묶을 때 패딩처리를 못하게 된다.

In [29]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서 에러남
                                 remove_columns=dataset["train"].column_names
                                 )

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

`convert_examples_to_features()`이 `dataset`의 모든 샘플에 다 적용되고 나면 `tokenized_datasets`는 다음처럼 됩니다.

In [30]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

기존에 있던 특성 `en`, `ko`는 사라졌고 `en`은 `input_ids`와 `attention_mask`로 `ko`는 `labels`로 바뀐것을 확인할 수 있습니다. 예를 들어 학습 세트에 10번 데이터를 보면 다음처럼 다 숫자라 바뀌게 된것이다.

In [31]:
tokenized_datasets['train'][10]

{'input_ids': [118,
  3292,
  33311,
  14,
  142,
  52398,
  2603,
  4,
  155,
  1237,
  28,
  3916,
  7,
  15015,
  4,
  187,
  6597,
  6832,
  20,
  5131,
  1171,
  22282,
  7095,
  909,
  2624,
  4834,
  13,
  615,
  67,
  2865,
  14348,
  20,
  13851,
  322,
  14,
  5,
  7095,
  909,
  2624,
  4834,
  2475,
  65,
  791,
  61114,
  4,
  5,
  2405,
  1019,
  3,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [386,
  1237,
  3916,
  44990,
  12193,
  6,
  4,
  2702,
  31054,
  4436,
  6895,
  360,
  35532,
  3841,
  3167,
  1308,
  2402,
  369,
  14084,
  82,
  61553,
  61,
  4,
  207,
  19358,
  22,
  439,
  4995,
  386,
  4488,
  35532,
  3841,
  3167,
  646,
  2707,
  41904,
  369,
  57841,
  12,
  179,
  160,
  1007,
  37,
  13017,
  3,
  1]}

토크나이저를 써서 숫자로 부터 토큰화 해보면 다음과 같습니다.

In [32]:
print( '원 데이터    :', dataset['train'][1]['en'] )
print( '처리 후 데이터:', tokenized_datasets['train'][1]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][1]['input_ids']) )

print('\n')
print( '원 데이터    :', dataset['train'][1]['ko'] )
print( '처리 후 데이터:', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][1]['labels']) )
print( '토큰화       :', tokenized_datasets['train'][1]['labels'] )

원 데이터    : so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.
처리 후 데이터: [262, 16, 10239, 16, 1051, 2520, 1590, 155, 16, 742, 36458, 1191, 16787, 54880, 45836, 3866, 20, 926, 742, 36458, 1191, 4, 16787, 45836, 3, 1]
토큰화       : ['▁so', '▁a', '▁mention', '▁a', '▁few', '▁weeks', '▁ago', '▁about', '▁a', '▁re', 'charge', 'able', '▁wireless', '▁optical', '▁mouse', '▁brought', '▁in', '▁another', '▁re', 'charge', 'able', ',', '▁wireless', '▁mouse', '.', '</s>']


원 데이터    : 모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.
처리 후 데이터: ['▁모든', '▁광', '마우스', '와', '▁마찬가지', '▁', '로', '▁이', '▁광', '마우스', '도', '▁책상', '▁위에', '▁놓는', '▁마우스', '▁', '패드', '를', '▁필요로', '▁하지', '▁않는다', '.', '</s>']
토큰화       : [350, 2908, 56715, 41, 10165, 7, 34, 47, 2908, 56715, 27, 14740, 2709, 30882, 50904, 7, 25963, 21, 11995, 751, 985, 3, 1]


데이터 특성화를 모두 마쳤으므로 이제 모델을 로딩하자. `AutoModelForSeq2SeqLM`를 사용해서 사전학습된 모델을 불러오면 사전학습된 T5모델 마지막에 파인튜닝할 수 있는 분류 헤드를 붙인 모델을 반환하게됩니다.

## Model

아래와 같이 모델을 로딩하고 모델 출력 시켜보면 T5 모델 레이어가 매우 길게 출력되는데 제일 마지막 부분에 다음과 같이 분류 헤드가 붙어 있는 것을 확인할 수 있습니다. 헤드를 보면 모델에서 출력하는 벡터는 768차원이고 이를 단어장 사이즈인 64128로 변환시키고 있는 것을 알 수 있습니다.

```
(lm_head): Linear(in_features=768, out_features=64128, bias=False)
```

이렇게 생성된 `model`은 인코더-디코더 구조를 가지는 트랜스포머이므로 이 모델을 포워딩 하려면 인코더 인풋과 디코더 인풋을 넣어줘야 합니다.


이때, 임의의 입력을 넣고 출력이 의도대로 나오는지 확인함으로써 모델이 제대로 작동 되는지 어떤 구조로 되어있는지 확인할 수 있습니다. 테스트를 하기위해 간단한 영어문장으로 테스트 해보겠습니다.



In [33]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [34]:
encoder_inputs = tokenizer(
    ["Studies have been shown that owning a dog is good for you"],
    return_tensors="pt"
)['input_ids'].to(device)

decoder_targets = tokenizer(
    ["개를 키우는 것이 건강에 좋다는 연구 결과가 있습니다."],
    return_tensors="pt"
)['input_ids'].to(device)


영어 문장은 인코더의 입력이 되고 한국어 문장은 디코더의 타겟이 됩니다. 아래처럼 모두 숫자로 변환되어 있습니다.

In [35]:
print( encoder_inputs )
print( decoder_targets )

tensor([[24611,    84,   166,  8135,    38,   847,    91,    16,  8146,    43,
           667,    40,   106,     1]], device='cuda:0')
tensor([[15833, 12236,   179, 16120, 28117,  1007,  3883,   327,     3,     1]],
       device='cuda:0')


이제 디코더 입력을 만들기위해 `model._shift_right`를 사용해 디코더 출력을 오른쪽으로 쉬프트 시킵니다.

In [36]:
decoder_inputs = model._shift_right(decoder_targets)

`decoder_inputs`와 `decoder_targets`이 어떻게 다른지 비교해보면

In [37]:
pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index=('decoder target', 'decoder input')
)

,0,1,2,3,4,5,6,7,8,9
decoder target,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.,</s>
decoder input,<pad>,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.


위처럼 오른쪽으로 쉬프트된 디코더 입력은 `<pad>` 토큰이 추가되었다. 이렇게 출력으로 쓰이는 문장을 오른쪽으로 쉬프트시켜 티처포싱Teacher forcing을 진행하게 됩니다. 다음처럼 `model`에 인코더 입력, 디코더 입력, 디코더 타겟을 입력하고 포워드 시킵니다.

In [38]:
# forward pass
outputs = model(input_ids=encoder_inputs,
                decoder_input_ids=decoder_inputs,
                labels=decoder_targets)

`model`의 `outputs`에는 다음과 같은 키가 있습니다.

In [39]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

손실함수 값을 다음처럼 확인할 수 있고 `grad_fn`이 있기 때문에 `output.loss`를 백워드 시킬 수 있다.

In [40]:
outputs.loss

tensor(87.8185, device='cuda:0', grad_fn=<NllLossBackward0>)

인코더의 마지막 상태는 (1, 14, 768)입니다. 각 숫자는 순서대로 샘플 수, 스탭 수, 모델 사이즈를 나타냅니다. 즉 인코더로 들어가는 14개 토큰이 각각 768차원 벡터로 인코딩되었습니다.

In [41]:
outputs['encoder_last_hidden_state'].shape

torch.Size([1, 14, 768])

`logit`은 디코더 입력 토큰 10개에 대한 그 다음 토큰 예측 10개를 담고있습니다. 샘플 한개에 대해서 10개 토큰에 대해서 64128개 단어에 대한 확률값이 들어 있습니다.

In [42]:
outputs['logits'].shape

torch.Size([1, 10, 64128])

`logit`에 `argmax`를 씌워서 토큰화시켜보면 다음과 같습니다.

In [43]:
tokenizer.convert_ids_to_tokens( torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() )

['큐브', '큐브', '▁비일비재', '▁비일비재', '▁베네', '▁비일비재', '▁베네', '▁베네', '큐브', '큐브']

마지막 헤더가 학습이 되지 않았기 때문에 적절한 아웃풋이 나오지 않지만 입력과 출력의 텐서 모양을 보면 포워드 패스가 제대로 작동한다는 것을 알 수 있습니다.

## Collator

파이토치에서 모델을 학습시키기 위해서 `DataLoader`를 사용하게 되는데 이 데이터 로더의 역할은 for 루프를 돌면서 데이터 셋으로 부터 샘플을 배치 수만큼 가져오는 것입니다. 이때 샘플을 배치 수만큼 무작위로 가져와 어떤 식으로든 각 샘플을 짝맞춤해서 반환해야하는데 서로 크기가 다른 샘플들을 다루는 경우는 길이를 맞춘다든지 패딩을 한다든지 하는 추가 작업이 필요하게 됩니다. 이런 작업이 일어나는 곳이 `collate_fn`으로 지정되는 함수입니다.

시퀀스 투 시퀀스 모델을 학습시킬때 이런 콜레이터 함수가 하는 전형적인 역할은 입력 또는 출력 문자열을 패딩하고 조금 전 모델에서 알아봤듯이 디코더 타겟을 오른쪽으로 한칸 쉬프트 시켜서 디코더 입력으로 만드는 것 입니다. 이런 작업을 자동으로 처리해주는 클래스가 `DataCollatorForSeq2Seq`입니다.

우선 콜레이터를 만들기 위해서는 토크나이저와 모델을 넘겨야 합니다.

In [44]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

앞서 만들어논 `tokenized_datasets`에서 샘플 두개를 조회하면 다음처럼 전체 결과는 사전으로 리턴되며 사전의 각 키 아래에 여러 샘플들의 값이 리스트로 들어있게 됩니다.

In [45]:
# 각 항목아래 샘플들이 리스트 형태로 묶여 반환된다.
print(tokenized_datasets["train"][1:3])

{'input_ids': [[262, 16, 10239, 16, 1051, 2520, 1590, 155, 16, 742, 36458, 1191, 16787, 54880, 45836, 3866, 20, 926, 742, 36458, 1191, 4, 16787, 45836, 3, 1], [8513, 174, 54880, 50453, 4, 341, 60, 187, 1686, 17, 71, 713, 16, 23899, 3, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[350, 2908, 56715, 41, 10165, 7, 34, 47, 2908, 56715, 27, 14740, 2709, 30882, 50904, 7, 25963, 21, 11995, 751, 985, 3, 1], [277, 4444, 360, 14740, 27, 11995, 751, 985, 3, 1]]}


콜레이터에 샘플을 넘길 때는 개별 샘플이 사전으로 묶이는 형태가 되어야 되므로 아래처럼 한번 가공하게 된다.

In [46]:
# 콜레이터에는 샘플을 개별 {}로 넘겨야 합니다.
[tokenized_datasets["train"][i] for i in range(1, 3)]

[{'input_ids': [262,
   16,
   10239,
   16,
   1051,
   2520,
   1590,
   155,
   16,
   742,
   36458,
   1191,
   16787,
   54880,
   45836,
   3866,
   20,
   926,
   742,
   36458,
   1191,
   4,
   16787,
   45836,
   3,
   1],
  'attention_mask': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  'labels': [350,
   2908,
   56715,
   41,
   10165,
   7,
   34,
   47,
   2908,
   56715,
   27,
   14740,
   2709,
   30882,
   50904,
   7,
   25963,
   21,
   11995,
   751,
   985,
   3,
   1]},
 {'input_ids': [8513,
   174,
   54880,
   50453,
   4,
   341,
   60,
   187,
   1686,
   17,
   71,
   713,
   16,
   23899,
   3,
   1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [277, 4444, 360, 14740, 27, 11995, 751, 985, 3, 1]}]

위에 반환된 결과를 보면 각 샘플이 사전 {}으로 묶이고 샘플 하나에는 `input_ids`, `attention_mask`, ` labels`이 존재합니다. 각 샘플을 리스트로 묶어서 콜레이터에게 전달하고 반환되는 값을 확인해보겠습니다.

In [47]:
# 콜레이터를 돌리면 알아서 패딩하고 쉬프트 시킵니다.
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


반환된 `batch`의 키를 확인해보면 `decoder_input_ids`가 생긴것을 확인할 수 있습니다.

In [48]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

`batch`의 각 키에 어떤 값들이 들어있는지 확인해봅니다.

In [49]:
batch

{'input_ids': tensor([[  262,    16, 10239,    16,  1051,  2520,  1590,   155,    16,   742,
         36458,  1191, 16787, 54880, 45836,  3866,    20,   926,   742, 36458,
          1191,     4, 16787, 45836,     3,     1],
        [ 8513,   174, 54880, 50453,     4,   341,    60,   187,  1686,    17,
            71,   713,    16, 23899,     3,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'labels': tensor([[  350,  2908, 56715,    41, 10165,     7,    34,    47,  2908, 56715,
            27, 14740,  2709, 30882, 50904,     7, 25963,    21, 11995,   751,
           985,     3,     1],
        [  277,  4444,   360, 14740,    27, 11995,   751,   985,     3,     1,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 

출력된 batch를 정리하면 아래처럼 된다.


새로 생긴 `decoder_input_ids`는 앞에 0(<pad>)이 붙어 있는 것이 보이고 `label`에서 끝에 1이 사라져 `label`이 오른쪽으로 쉬프트된 것임을 알 수 있습니다. 그리고 또 두번째 샘플 `labels`에서 마지막에 -100 이 보인다. 이 값은 `label`이 패딩된 것을 나타내며 손실 함수값을 계산할 때 -100이 있는 위치는 손실을 계산하지 않게 됩니다. 이렇게 시퀀스 투 시퀀스 모델을 학습하기 위해 필요한 자잘한 작업을 콜레이터가 알아서 자동으로 처리합니다.
    
   
